# get runs

In [4]:
import numpy as np

In [5]:
import pandas as pd 
import wandb
api = wandb.Api()

# Project is specified by <entity/project-name>
# runs = api.runs("mines-paristech-cmm/unetdd-mvtec-00")

# sweep = api.sweep(path/to/sweep)
sweep = api.sweep("mines-paristech-cmm/unetdd-mvtec-00/7e6qg2bj")
runs = sweep.runs

In [6]:
summary_list, config_list, name_list = [], [], []
tags_list = []
ids_list = []
state_list = []

for run in runs: 
    summary_list.append(run.summary._json_dict)
    config_list.append(
        {k: v for k,v in run.config.items()
          if not k.startswith('_')})
    name_list.append(run.name)
    tags_list.append(run.tags)
    ids_list.append(run.id)
    state_list.append(run.state)
    
runs_df = pd.DataFrame({
    "summary": summary_list,
    "config": config_list,
    "name": name_list,
    'tags': tags_list,
    'id': ids_list,
    'state': state_list,
})

In [7]:
runs_df.shape

(292, 6)

In [9]:
runs_df.columns

Index(['summary', 'config', 'name', 'tags', 'id', 'state'], dtype='object')

# filter runs

In [10]:
runs_df_filtered = runs_df

get_only_finished = runs_df['state'] == 'finished'
runs_df_filtered = runs_df_filtered[get_only_finished]

select_epoch_is_last = runs_df["summary"].apply(lambda s: s.get("epoch", None)) == 600
runs_df_filtered = runs_df_filtered[select_epoch_is_last]

runs_df_filtered.shape

/tmp/ipykernel_24481/2387820682.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  runs_df_filtered = runs_df_filtered[select_epoch_is_last]


(183, 6)

In [11]:
runs_df_filtered["summary"].iloc[0].keys()


dict_keys(['_timestamp', 'validate/loss', 'test/loss-anomalous-mean', '_step', 'normal_class_label', 'test/score-normals-mean', 'train/loss-normals-mean', '_runtime', 'train/loss', 'test/loss-normals-mean', 'validate/avg-precision', 'train/loss-anomalous-mean', 'train/pixel-count-anomalous', 'validate/pixel-count-anomalous', 'test/loss', 'train/score-mean', 'validate/score-mean', 'test/images-heatmaps-table', 'epoch', '_wandb', 'validate/roc-auc', 'test/roc-curve_table', 'test/score-anomalous-mean', 'validate/loss-anomalous-mean', 'test/avg-precision', 'trainer/global_step', 'batch_size_effective', 'test/pixel-count-normal', 'normal_class_fullqualified', 'validate/loss-normals-mean', 'validate/score-anomalous-mean', 'validate/images-heatmaps-table', 'test/roc-auc', 'test/score-mean', 'mvtec_class_type', 'train/pixel-count-normal', 'train/score-normals-mean', 'test/pixel-count-anomalous', 'train/score-anomalous-mean', 'validate/pixel-count-normal', 'validate/score-normals-mean', 'test/p

In [12]:
runs_df_filtered["config"].iloc[0].keys()

dict_keys(['it', 'lr', 'pid', 'loss', 'root', 'seed', 'test', 'model', 'epochs', 'logdir', 'datadir', 'dataset', 'in_shape', 'nworkers', 'loss_name', 'net_shape', 'optimizer', 'raw_shape', 'scheduler', 'seeds_str', 'batch_size', 'model_name', 'pin_memory', 'wandb_watch', 'normal_class', 'slurm_job_id', 'wandb_log_pr', 'weight_decay', 'learning_rate', 'preprocessing', 'wandb_log_roc', 'wandb_profile', 'optimizer_name', 'scheduler_name', 'script_version', 'slurm_job_gpus', 'slurm_job_name', 'slurm_job_user', 'slurm_task_pid', 'supervise_mode', 'confighash_full', 'preview_nimages', 'slurmd_nodename', 'confighash_slurm', 'script_start_time', 'slurm_submit_host', 'lightning_ndevices', 'lightning_profiler', 'lightning_strategy', 'loss_stage_weights', 'real_anomaly_limit', 'slurm_array_job_id', 'slurm_cluster_name', 'lightning_precision', 'slurm_array_task_id', 'slurm_job_partition', 'scheduler_parameters', 'wandb_watch_log_freq', 'lightning_accelerator', 'loss_scoremap_strategy', 'lightning_

# add metric to  a single run

In [13]:
run = runs_df_filtered.iloc[0]
runid = run['id']
run = api.run(f"mines-paristech-cmm/unetdd-mvtec-00/{run['id']}")

In [14]:
run.history(keys=["validate/avg-precision"], samples=600).T

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
_step,10.000000,22.000000,34.000000,46.000000,58.000000,70.000000,82.00000,94.000000,106.000000,118.000000,...,610.000000,622.000000,634.000000,646.000000,658.000000,670.000000,682.00000,694.000000,706.000000,718.000000
validate/avg-precision,0.293818,0.114076,0.194144,0.233523,0.295656,0.206246,0.29022,0.333758,0.259987,0.328019,...,0.048452,0.106554,0.038347,0.098874,0.133136,0.107699,0.15804,0.019767,0.061726,0.078268


In [15]:
metric_history = np.array(list(dic["validate/avg-precision"] for dic in run.scan_history(keys=["validate/avg-precision"])))
assert metric_history.shape == (60,), f"Expected shape (600,), got {metric_history.shape}"
metric_history_accumulated = metric_history.sum()
metric_history_accumulated

6.074131992384064

In [16]:
run.summary.update({"avg-precision-accumulated": metric_history_accumulated})

In [17]:
run = runs_df_filtered.iloc[0]
runid = run['id']
run = api.run(f"mines-paristech-cmm/unetdd-mvtec-00/{runid}")
run.summary["avg-precision-accumulated"]

6.074131992384064

# apply to all sweep runs

In [18]:
import progressbar

def update_summary_with_accumulated_metric(runid: str, metric_key: str):
    run = api.run(f"mines-paristech-cmm/unetdd-mvtec-00/{runid}")
    metric_history = np.array(list(dic[metric_key] for dic in run.scan_history(keys=[metric_key])))
    assert metric_history.shape == (60,), f"Expected shape (600,), got {metric_history.shape}"
    metric_history_accumulated = metric_history.sum()
    metric_history_mean = metric_history.mean()
    run.summary.update({
        f"{metric_key}/history-accumulated": metric_history_accumulated,
        f"{metric_key}/history-mean": metric_history_mean,
    })
    run.save()

for rowidx, row in progressbar.progressbar(runs_df_filtered.iterrows(), max_value=runs_df_filtered.shape[0]):
    update_summary_with_accumulated_metric(row['id'], "validate/avg-precision")

100% (183 of 183) |######################| Elapsed Time: 0:04:22 Time:  0:04:22
